In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/STORM_preprocessed_medianfill_1.csv', index_col=0) # 200 column
evaluate_dict = dict()

In [5]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, mean_absolute_error
import numpy as np
from typing import Literal


def evaluate(model, X_test, y_test, threshold: float = None,
             mode: Literal["classification", "regression"] = "classification") -> dict:
    """
    Evaluates the performance of a scikit-learn model (binary classification or regression).

    Args:
        model (object): The trained model (with a predict_proba method for classification).
        X_test (array-like): The input features for testing.
        y_test (array-like): The true labels or values for testing.
        threshold (float): For classification, the probability threshold. Default is 0.5.
                           For regression, the allowed error threshold. Default is 0.2 (20%).
        mode (Literal["classification", "regression"]): The evaluation mode.
            - "classification": Confusion matrix, accuracy, precision, recall.
            - "regression": MAE, MSE, RMSE with optional tolerance check.

    Returns:
        dict: A dictionary with relevant evaluation metrics.

    Raises:
        ValueError: If the mode is invalid or threshold is not in the valid range.
    """
    if threshold is None:
        threshold = 0.5 if mode == "classification" else 0.2

    if not (0 <= threshold <= 1):
        raise ValueError("Threshold must be between 0 and 1.")

    if mode == "classification":
        y_prob = model.predict_proba(X_test)[:, 1]  # Probability of class 1
        y_pred = (y_prob >= threshold).astype(int)

        cm = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)

        return {
            'confusion_matrix': cm,
            'accuracy': round(accuracy, 2),
            'precision': round(precision, 2),
            'recall': round(recall, 2)
        }

    elif mode == "regression":
        y_pred = model.predict(X_test)

        mae = mean_absolute_error(y_test, y_pred)
        mse = np.mean((y_test - y_pred) ** 2)
        rmse = np.sqrt(mse)

        # Optional: Check if errors are within the tolerance threshold
        tolerance_mae = threshold * np.mean(np.abs(y_test))
        tolerance_rmse = threshold * np.std(y_test)
        tolerance_mse = threshold * np.var(y_test)

        within_tolerance = {
            'mae_upperbound_tolerance': round(tolerance_mae - mae, 2),
            'rmse_upperbound_tolerance': round(tolerance_rmse - rmse, 2),
            'mse_upperbound_tolerance': round(tolerance_mse - mse, 2)
        }

        return {
            'mae': round(mae, 2),
            'mse': round(mse, 2),
            'rmse': round(rmse, 2),
            **within_tolerance
        }

    else:
        raise ValueError("Invalid mode. Mode must be 'classification' or 'regression'.")



Mạng Residual (ResNet) lý tưởng cho các mô hình sâu, ngăn chặn hiện tượng mất mát gradient.

Các lớp Dropout có thể giúp ngăn ngừa hiện tượng quá khớp (overfitting).

TabNet là một mô hình tiên tiến được thiết kế đặc biệt cho dữ liệu bảng (tabular data), tận dụng các cơ chế attention.

**1. Target 1 : TotalDeaths**

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

LINEAR_TARGETS = ["TotalDeaths", "NoInjured", "TotalDamageAdjusted(000US$)"]
ATTRIBUTES = ['Year', 'Month', 'MainLandfallLocation', 'OFDAResponse', 'Appeal', 'Declaration', 'LandfallMagnitude(kph)', 'LandfallPressure(mb)']
CATEGORICAL_TARGETS = ['Flood', 'Slide']

X = df[ATTRIBUTES + CATEGORICAL_TARGETS]
y = df[LINEAR_TARGETS[0]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**TabNet**

In [11]:
import numpy as np
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error

# Convert data to NumPy arrays (if they are not already)
X_train_np = X_train_scaled
X_test_np = X_test_scaled
y_train_np = y_train.values.reshape(-1, 1)
y_test_np = y_test.values.reshape(-1, 1)

# Define the TabNet Regressor
tabnet_model = TabNetRegressor()

# Train the model with verbose set to 0
tabnet_model.fit(
    X_train_np, y_train_np,
    eval_set=[(X_test_np, y_test_np)],
    eval_metric=['rmse'],
    max_epochs=100,
    patience=100,
    batch_size=32,
    virtual_batch_size=8
)

c:\Users\anhmi\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 266942.05219| val_0_rmse: 612.55852|  0:00:00s
epoch 1  | loss: 269927.17511| val_0_rmse: 612.05807|  0:00:00s
epoch 2  | loss: 270525.38898| val_0_rmse: 611.62574|  0:00:00s
epoch 3  | loss: 224111.95514| val_0_rmse: 611.20167|  0:00:00s
epoch 4  | loss: 267574.88184| val_0_rmse: 610.59015|  0:00:01s
epoch 5  | loss: 270124.41931| val_0_rmse: 610.20462|  0:00:01s
epoch 6  | loss: 265885.11882| val_0_rmse: 608.56339|  0:00:01s
epoch 7  | loss: 61682.90042| val_0_rmse: 607.28727|  0:00:01s
epoch 8  | loss: 266861.51843| val_0_rmse: 606.17711|  0:00:01s
epoch 9  | loss: 260414.26965| val_0_rmse: 600.03537|  0:00:01s
epoch 10 | loss: 263199.93756| val_0_rmse: 602.45385|  0:00:01s
epoch 11 | loss: 254101.51715| val_0_rmse: 598.4637|  0:00:02s
epoch 12 | loss: 59872.81744| val_0_rmse: 594.13894|  0:00:02s
epoch 13 | loss: 251529.24976| val_0_rmse: 592.14775|  0:00:02s
epoch 14 | loss: 257262.80566| val_0_rmse: 589.71154|  0:00:02s
epoch 15 | loss: 254115.43481| val_0_rmse: 

c:\Users\anhmi\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [12]:
eval_values = evaluate(tabnet_model, X_test_np, y_test_np, threshold=0.3, mode="regression")
evaluate_dict = {}
evaluate_dict["TabNet"] = eval_values

eval_values

{'mae': 146.92,
 'mse': 334204.99,
 'rmse': 578.1,
 'mae_upperbound_tolerance': -102.76,
 'rmse_upperbound_tolerance': -399.52,
 'mse_upperbound_tolerance': -227897.9}

**ResNet**

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
def residual_block(x, units):
    shortcut = x
    x = layers.Dense(units, activation='relu')(x)
    x = layers.Dense(units)(x)  # No activation for the second layer
    x = layers.add([x, shortcut])  # Add the shortcut
    x = layers.Activation('relu')(x)
    return x


def build_resnet(input_shape, output_units):
    inputs = keras.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)

    # Add several residual blocks
    for _ in range(3):  # Adjust the number of blocks as needed
        x = residual_block(x, 64)

    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(output_units)(x)  # For regression, no activation here

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

resnet_model = build_resnet(input_shape=(X_train_scaled.shape[1],), output_units=1)
resnet_model.compile(optimizer='adam', loss='mean_squared_error')

checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", mode="min")

resnet_model.fit(X_train_scaled, y_train_np, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test_np), callbacks=[checkpoint], verbose=0)


In [15]:
eval_values = evaluate(resnet_model, X_test_scaled, y_test_np, threshold=0.3, mode="regression")
evaluate_dict["ResNet"] = eval_values

eval_values

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


{'mae': 141.08,
 'mse': 350334.57,
 'rmse': 591.89,
 'mae_upperbound_tolerance': -96.93,
 'rmse_upperbound_tolerance': -413.31,
 'mse_upperbound_tolerance': -244027.48}

**LSTM**

In [139]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [140]:
# Reshape the data for LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y.iloc[i + time_steps])  # Corresponding y value
    return np.array(Xs), np.array(ys)

TIME_STEPS = 1  # You can change this value based on your needs
X_train_lstm, y_train_lstm = create_dataset(pd.DataFrame(X_train_scaled), pd.Series(y_train), TIME_STEPS)
X_test_lstm, y_test_lstm = create_dataset(pd.DataFrame(X_test_scaled), pd.Series(y_test), TIME_STEPS)

# Reshape input to be [samples, time steps, features]
X_train_lstm = X_train_lstm.reshape((X_train_lstm.shape[0], X_train_lstm.shape[1], X_train_lstm.shape[2]))
X_test_lstm = X_test_lstm.reshape((X_test_lstm.shape[0], X_test_lstm.shape[1], X_test_lstm.shape[2]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer for regression (adjust this based on the number of targets)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_lstm, y_train_lstm, epochs=100, batch_size=32, validation_data=(X_test_lstm, y_test_lstm), verbose=0)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [141]:
eval_values = evaluate(model, X_test_lstm, y_test_lstm, threshold=0.3, mode="regression")
evaluate_dict["LSTM"] = eval_values

eval_values

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 473ms/step


{'mae': 144.85,
 'mse': 381573.82,
 'rmse': 617.72,
 'mae_upperbound_tolerance': -100.35,
 'rmse_upperbound_tolerance': -436.68,
 'mse_upperbound_tolerance': -272328.29}

In [142]:
# compare metrics value
def highlight_max(s):
    is_max = s == s.max()
    return ['color: red' if v else '' for v in is_max]

def highlight_min(s):
    is_min = s == s.min()
    return ['color: red' if v else '' for v in is_min]

def highlight_row(row, selected_method):
    return ['background-color: black;' if row['Method'] in selected_method else ''
            for _ in row]

selected_method = [model.__class__.__name__]
eval_value_df = pd.DataFrame(evaluate_dict).T.reset_index().rename(columns={"index":"Method"})

eval_value_df = (
    eval_value_df.style
    .apply(highlight_max, subset=["mae_upperbound_tolerance", "rmse_upperbound_tolerance", "mse_upperbound_tolerance"])
    .apply(highlight_min, subset=["mae", "mse", "rmse"])
    .apply(lambda row: highlight_row(row, selected_method), axis=1 )
    .format(precision=2)
)

eval_value_df

,Method,mae,mse,rmse,mae_upperbound_tolerance,rmse_upperbound_tolerance,mse_upperbound_tolerance
0,TabNet,151.39,356207.52,596.83,-107.23,-418.25,-249900.43
1,ResNet,159.28,375273.13,612.60,-115.13,-434.01,-268966.04
2,Wide & Deep Model,162.67,388620.51,623.39,-118.51,-444.81,-282313.42
3,LSTM,144.85,381573.82,617.72,-100.35,-436.68,-272328.29


**2. Target 2 : NoInjured**

In [143]:
LINEAR_TARGETS = ["TotalDeaths", "NoInjured", "TotalDamageAdjusted(000US$)"]
ATTRIBUTES = ['Year', 'Month', 'MainLandfallLocation', 'OFDAResponse', 'Appeal', 'Declaration', 'LandfallMagnitude(kph)', 'LandfallPressure(mb)']
CATEGORICAL_TARGETS = ['Flood', 'Slide']

X = df[ATTRIBUTES + CATEGORICAL_TARGETS]
y = df[LINEAR_TARGETS[1]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**TabNet**

In [144]:
import numpy as np
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error

# Convert data to NumPy arrays (if they are not already)
X_train_np = X_train_scaled
X_test_np = X_test_scaled
y_train_np = y_train.values.reshape(-1, 1)
y_test_np = y_test.values.reshape(-1, 1)

# Define the TabNet Regressor
tabnet_model = TabNetRegressor()

# Train the model with verbose set to 0
tabnet_model.fit(
    X_train_np, y_train_np,
    eval_set=[(X_test_np, y_test_np)],
    eval_metric=['rmse'],
    max_epochs=100,
    patience=100,
    batch_size=32,
    virtual_batch_size=8
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 41342.91809| val_0_rmse: 457.48739|  0:00:00s
epoch 1  | loss: 40483.16895| val_0_rmse: 457.41892|  0:00:00s
epoch 2  | loss: 30989.36633| val_0_rmse: 456.37871|  0:00:00s
epoch 3  | loss: 39795.71899| val_0_rmse: 454.79109|  0:00:00s
epoch 4  | loss: 38749.64453| val_0_rmse: 453.96837|  0:00:00s
epoch 5  | loss: 37853.73267| val_0_rmse: 453.16265|  0:00:00s
epoch 6  | loss: 38109.79175| val_0_rmse: 450.33838|  0:00:01s
epoch 7  | loss: 37540.91101| val_0_rmse: 451.09755|  0:00:01s
epoch 8  | loss: 35628.43921| val_0_rmse: 449.84134|  0:00:01s
epoch 9  | loss: 23545.63745| val_0_rmse: 448.97983|  0:00:01s
epoch 10 | loss: 35801.52747| val_0_rmse: 446.69131|  0:00:01s
epoch 11 | loss: 21865.74988| val_0_rmse: 444.70022|  0:00:01s
epoch 12 | loss: 34280.8114| val_0_rmse: 439.59275|  0:00:01s
epoch 13 | loss: 33762.63684| val_0_rmse: 436.90352|  0:00:02s
epoch 14 | loss: 32441.63953| val_0_rmse: 432.77404|  0:00:02s
epoch 15 | loss: 32575.72107| val_0_rmse: 433.4502|  0:0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [145]:
eval_values = evaluate(tabnet_model, X_test_np, y_test_np, threshold=0.3, mode="regression")
evaluate_dict = {}
evaluate_dict["TabNet"] = eval_values

eval_values

{'mae': 134.44,
 'mse': 177873.72,
 'rmse': 421.75,
 'mae_upperbound_tolerance': -93.88,
 'rmse_upperbound_tolerance': -290.38,
 'mse_upperbound_tolerance': -120345.95}

**ResNet**

In [146]:
def residual_block(x, units):
    shortcut = x
    x = layers.Dense(units, activation='relu')(x)
    x = layers.Dense(units)(x)  # No activation for the second layer
    x = layers.add([x, shortcut])  # Add the shortcut
    x = layers.Activation('relu')(x)
    return x


def build_resnet(input_shape, output_units):
    inputs = keras.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)

    # Add several residual blocks
    for _ in range(3):  # Adjust the number of blocks as needed
        x = residual_block(x, 64)

    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(output_units)(x)  # For regression, no activation here

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

resnet_model = build_resnet(input_shape=(X_train_scaled.shape[1],), output_units=1)
resnet_model.compile(optimizer='adam', loss='mean_squared_error')

checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", mode="min")

resnet_model.fit(X_train_scaled, y_train_np, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test_np), callbacks=[checkpoint], verbose=0)

In [147]:
eval_values = evaluate(resnet_model, X_test_scaled, y_test_np, threshold=0.3, mode="regression")
evaluate_dict["ResNet"] = eval_values

eval_values

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step


{'mae': 152.35,
 'mse': 202205.47,
 'rmse': 449.67,
 'mae_upperbound_tolerance': -111.8,
 'rmse_upperbound_tolerance': -318.3,
 'mse_upperbound_tolerance': -144677.7}

**LSTM**

In [150]:
# Reshape the data for LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y.iloc[i + time_steps])  # Corresponding y value
    return np.array(Xs), np.array(ys)

TIME_STEPS = 1  # You can change this value based on your needs
X_train_lstm, y_train_lstm = create_dataset(pd.DataFrame(X_train_scaled), pd.Series(y_train), TIME_STEPS)
X_test_lstm, y_test_lstm = create_dataset(pd.DataFrame(X_test_scaled), pd.Series(y_test), TIME_STEPS)

# Reshape input to be [samples, time steps, features]
X_train_lstm = X_train_lstm.reshape((X_train_lstm.shape[0], X_train_lstm.shape[1], X_train_lstm.shape[2]))
X_test_lstm = X_test_lstm.reshape((X_test_lstm.shape[0], X_test_lstm.shape[1], X_test_lstm.shape[2]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer for regression (adjust this based on the number of targets)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_lstm, y_train_lstm, epochs=100, batch_size=32, validation_data=(X_test_lstm, y_test_lstm), verbose=0)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [151]:
eval_values = evaluate(model, X_test_lstm, y_test_lstm, threshold=0.3, mode="regression")
evaluate_dict["LSTM"] = eval_values

eval_values

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 276ms/step


{'mae': 133.08,
 'mse': 211609.92,
 'rmse': 460.01,
 'mae_upperbound_tolerance': -91.56,
 'rmse_upperbound_tolerance': -326.96,
 'mse_upperbound_tolerance': -152597.8}

In [152]:
# compare metrics value
def highlight_max(s):
    is_max = s == s.max()
    return ['color: red' if v else '' for v in is_max]

def highlight_min(s):
    is_min = s == s.min()
    return ['color: red' if v else '' for v in is_min]

def highlight_row(row, selected_method):
    return ['background-color: black;' if row['Method'] in selected_method else ''
            for _ in row]

selected_method = [model.__class__.__name__]
eval_value_df = pd.DataFrame(evaluate_dict).T.reset_index().rename(columns={"index":"Method"})

eval_value_df = (
    eval_value_df.style
    .apply(highlight_max, subset=["mae_upperbound_tolerance", "rmse_upperbound_tolerance", "mse_upperbound_tolerance"])
    .apply(highlight_min, subset=["mae", "mse", "rmse"])
    .apply(lambda row: highlight_row(row, selected_method), axis=1 )
    .format(precision=2)
)

eval_value_df

,Method,mae,mse,rmse,mae_upperbound_tolerance,rmse_upperbound_tolerance,mse_upperbound_tolerance
0,TabNet,134.44,177873.72,421.75,-93.88,-290.38,-120345.95
1,ResNet,152.35,202205.47,449.67,-111.80,-318.30,-144677.70
2,Wide & Deep Model,131.23,176558.92,420.19,-90.68,-288.82,-119031.14
3,LSTM,133.08,211609.92,460.01,-91.56,-326.96,-152597.80


**3. Target 3 : TotalDamageAdjusted(000US$)**

In [153]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

LINEAR_TARGETS = ["TotalDeaths", "NoInjured", "TotalDamageAdjusted(000US$)"]
ATTRIBUTES = ['Year', 'Month', 'MainLandfallLocation', 'OFDAResponse', 'Appeal', 'Declaration', 'LandfallMagnitude(kph)', 'LandfallPressure(mb)']
CATEGORICAL_TARGETS = ['Flood', 'Slide']

X = df[ATTRIBUTES + CATEGORICAL_TARGETS]
y = df[LINEAR_TARGETS[2]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**TabNet**

In [154]:
import numpy as np
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error

# Convert data to NumPy arrays (if they are not already)
X_train_np = X_train_scaled
X_test_np = X_test_scaled
y_train_np = y_train.values.reshape(-1, 1)
y_test_np = y_test.values.reshape(-1, 1)

# Define the TabNet Regressor
tabnet_model = TabNetRegressor()

# Train the model with verbose set to 0
tabnet_model.fit(
    X_train_np, y_train_np,
    eval_set=[(X_test_np, y_test_np)],
    eval_metric=['rmse'],
    max_epochs=100,
    patience=100,
    batch_size=32,
    virtual_batch_size=8
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 59933203456.0| val_0_rmse: 286303.09674|  0:00:00s
epoch 1  | loss: 56422007808.0| val_0_rmse: 286302.05232|  0:00:00s
epoch 2  | loss: 57089082368.0| val_0_rmse: 286300.35414|  0:00:00s
epoch 3  | loss: 52097675008.0| val_0_rmse: 286299.0372|  0:00:01s
epoch 4  | loss: 55271347200.0| val_0_rmse: 286297.78731|  0:00:01s
epoch 5  | loss: 58143604736.0| val_0_rmse: 286291.09494|  0:00:01s
epoch 6  | loss: 57428844032.0| val_0_rmse: 286291.52883|  0:00:01s
epoch 7  | loss: 56114516096.0| val_0_rmse: 286286.36255|  0:00:01s
epoch 8  | loss: 59089842688.0| val_0_rmse: 286285.31364|  0:00:01s
epoch 9  | loss: 50765489152.0| val_0_rmse: 286284.26121|  0:00:02s
epoch 10 | loss: 53528266752.0| val_0_rmse: 286281.04692|  0:00:02s
epoch 11 | loss: 42522254336.0| val_0_rmse: 286279.59562|  0:00:02s
epoch 12 | loss: 55818397696.0| val_0_rmse: 286272.00518|  0:00:02s
epoch 13 | loss: 45203825152.0| val_0_rmse: 286275.65147|  0:00:02s
epoch 14 | loss: 59033167872.0| val_0_rmse: 28627

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [155]:
eval_values = evaluate(tabnet_model, X_test_np, y_test_np, threshold=0.3, mode="regression")
evaluate_dict = {}
evaluate_dict["TabNet"] = eval_values

eval_values

{'mae': 122286.02,
 'mse': 81025980486.77,
 'rmse': 284650.63,
 'mae_upperbound_tolerance': -85179.62,
 'rmse_upperbound_tolerance': -207188.13,
 'mse_upperbound_tolerance': -61024519290.22}

**ResNet**

In [156]:
def residual_block(x, units):
    shortcut = x
    x = layers.Dense(units, activation='relu')(x)
    x = layers.Dense(units)(x)  # No activation for the second layer
    x = layers.add([x, shortcut])  # Add the shortcut
    x = layers.Activation('relu')(x)
    return x


def build_resnet(input_shape, output_units):
    inputs = keras.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)

    # Add several residual blocks
    for _ in range(3):  # Adjust the number of blocks as needed
        x = residual_block(x, 64)

    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(output_units)(x)  # For regression, no activation here

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

resnet_model = build_resnet(input_shape=(X_train_scaled.shape[1],), output_units=1)
resnet_model.compile(optimizer='adam', loss='mean_squared_error')

checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", mode="min")

resnet_model.fit(X_train_scaled, y_train_np, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test_np), callbacks=[checkpoint], verbose=0)

In [157]:
eval_values = evaluate(resnet_model, X_test_scaled, y_test_np, threshold=0.3, mode="regression")
evaluate_dict["ResNet"] = eval_values

eval_values

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step


{'mae': 101821.06,
 'mse': 28939500423.19,
 'rmse': 170116.14,
 'mae_upperbound_tolerance': -64714.66,
 'rmse_upperbound_tolerance': -92653.64,
 'mse_upperbound_tolerance': -8938039226.63}

**LSTM**

In [160]:
# Reshape the data for LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y.iloc[i + time_steps])  # Corresponding y value
    return np.array(Xs), np.array(ys)

TIME_STEPS = 1  # You can change this value based on your needs
X_train_lstm, y_train_lstm = create_dataset(pd.DataFrame(X_train_scaled), pd.Series(y_train), TIME_STEPS)
X_test_lstm, y_test_lstm = create_dataset(pd.DataFrame(X_test_scaled), pd.Series(y_test), TIME_STEPS)

# Reshape input to be [samples, time steps, features]
X_train_lstm = X_train_lstm.reshape((X_train_lstm.shape[0], X_train_lstm.shape[1], X_train_lstm.shape[2]))
X_test_lstm = X_test_lstm.reshape((X_test_lstm.shape[0], X_test_lstm.shape[1], X_test_lstm.shape[2]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer for regression (adjust this based on the number of targets)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_lstm, y_train_lstm, epochs=100, batch_size=32, validation_data=(X_test_lstm, y_test_lstm), verbose=0)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [161]:
eval_values = evaluate(model, X_test_lstm, y_test_lstm, threshold=0.3, mode="regression")
evaluate_dict["LSTM"] = eval_values

eval_values

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 707ms/step


{'mae': 125518.38,
 'mse': 84150972646.34,
 'rmse': 290087.87,
 'mae_upperbound_tolerance': -87856.27,
 'rmse_upperbound_tolerance': -211629.66,
 'mse_upperbound_tolerance': -63632002088.16}

In [162]:
# compare metrics value
def highlight_max(s):
    is_max = s == s.max()
    return ['color: red' if v else '' for v in is_max]

def highlight_min(s):
    is_min = s == s.min()
    return ['color: red' if v else '' for v in is_min]

def highlight_row(row, selected_method):
    return ['background-color: black;' if row['Method'] in selected_method else ''
            for _ in row]

selected_method = [model.__class__.__name__]
eval_value_df = pd.DataFrame(evaluate_dict).T.reset_index().rename(columns={"index":"Method"})

eval_value_df = (
    eval_value_df.style
    .apply(highlight_max, subset=["mae_upperbound_tolerance", "rmse_upperbound_tolerance", "mse_upperbound_tolerance"])
    .apply(highlight_min, subset=["mae", "mse", "rmse"])
    .apply(lambda row: highlight_row(row, selected_method), axis=1 )
    .format(precision=2)
)

eval_value_df

,Method,mae,mse,rmse,mae_upperbound_tolerance,rmse_upperbound_tolerance,mse_upperbound_tolerance
0,TabNet,122286.02,81025980486.77,284650.63,-85179.62,-207188.13,-61024519290.22
1,ResNet,101821.06,28939500423.19,170116.14,-64714.66,-92653.64,-8938039226.63
2,Wide & Deep Model,105283.89,40908552925.02,202258.63,-68177.49,-124796.13,-20907091728.47
3,LSTM,125518.38,84150972646.34,290087.87,-87856.27,-211629.66,-63632002088.16


TabNet hoạt động tốt với target 1, Wide & Deep Model	tốt trên target 2, trong khi ResNet có kết quả tốt nhất trên target 3